# Introdução

Considerando um cenário corporativo, a maior parte dos dados que um analista de dados tem para trabalhar já estão nos data warehouses, lake houses e afins, para que o mesmo esteja respeitando as políticas de data management e compliance. Mas o cenário se modifica quando vamos para empresas menores, que tem uma politica mais liberal de data management: é comum que os dados nos lakes precisem ser enriquecidos pelo próprio analista com arquivos de texto, planilhas, APIs ou até mesmo de páginas web enquanto tentam criar as visualizações.

A partir do momento em que os dados estão disponíveis, o próximo passo é trabalha-los para que os mesmos atendam às necessidades do projeto. Nesta fase, a biblioteca `Pandas` é uma das mais utilizadas para manipulação de dados em Python. `Pandas` é uma biblioteca de código aberto que fornece estruturas de dados de alto desempenho e fáceis de usar, além de ferramentas de análise de dados. Se considerarmos um analista que não tem background em programação, `Pandas` é uma excelente opção para manipulação de dados.

Para aqueles analistas com background em programação, apesar do poder que `Pandas` disponibiliza, sabemos que a maior parte das transformações de dados poderiam ser feitas de maneira mais tradicional e eficiente com o uso de `SQL`, que é a linguagem nativa dos bancos de dados relacionais e especializada para trabalhar com dados tabulares.

Mas, devido a algumas limitações, usar `SQL` nesta tarefa acaba não sendo tão popular. Entre os problemas mais comuns estão:

- Os ambientes de desenvolvimento em SQL não tem integração nativa com ferramentas de source control, como Git
- Não é possível executar testes automatizados de maneira simples
- Não é possível reutilizar código de maneira eficiente.

Para resolver esses problemas e outros relacionados a modelagem e transformação de dados, é que a ferramenta `dbt` foi criada.

Nesta série de artigos, vamos explorar o `dbt` e como ele pode ser utilizado para transformar dados de maneira eficiente e escalável. Para isso, vamos criar um projeto de exemplo: um repositório de dados para ser utilizado em visualizações de dados em geral. Este projeto pode ser um bom ponto de partida para quem está começando a trabalhar com `dbt` e quer adicionar alfo prático ao seu portfolio.

# dbt (data build tool)

O [dbt](https://www.getdbt.com/product/what-is-dbt) é uma ferramenta de transformação de dados que permite que você escreva transformações de dados em `SQL`, e que você possa executar essas transformações de maneira programática, testar e documentar essas transformações.

O dbt vêm em duas versões: `dbt Core` e `dbt Cloud`. A versão `dbt Core` é a versão open source, que você pode instalar e rodar em sua máquina. A versão `dbt Cloud` é a versão paga, que roda na nuvem e que oferece funcionalidades adicionais, como agendamento de execuções, monitoramento de execuções, alertas, etc.

Este artigo irá focar na versão `dbt Core`.

## Instalação

O dbt-core é uma ferramenta de linha de comando, e mais objetivamente, um pacote Python. Para instalar o dbt, você precisa ter o Python instalado em sua máquina. Para instalar o dbt, você pode usar o `pip`, o gerenciador de pacotes do Python. Para instalar o dbt, execute o seguinte comando:

```bash
pip install dbt
```
Após a instalação, você pode verificar se o dbt foi instalado corretamente executando o seguinte comando:

```bash
dbt --version
```
A saída do comando acima deve ser algo como:

```bash
dbt version: 0.21.0
```
A próxima etapa é baixar o driver de conexão com o banco de dados que você irá utilizar. O dbt suporta diversos bancos de dados, como `PostgreSQL`, `Redshift`, `BigQuery`, `Snowflake`, `SQL Server`, `MySQL`, `SQLite`, entre outros. O driver de conexão com o banco de dados que vamos utilizar aqui é do `PostgreSQL`, e você pode executar o seguinte comando para instala-lo:

```bash
pip install dbt-postgres
```

## Configuração

Agora que você já instalou o dbt e o driver de conexão com o banco de dados, o próximo passo é configurar o dbt para que ele possa se conectar ao seu banco de dados. Para isso, você precisa criar um arquivo chamado `profiles.yml`. Este arquivo pode estar localizado em dois locais:

- na pasta `~/.dbt/` no linux ou mac
- na pasta do projeto. --> usaremos essa opção, para mantermos todos os arquivos em um único local. 

O arquivo `profiles.yml` é um arquivo de configuração que contém as informações de conexão com o banco de dados. Abaixo está um exemplo de um arquivo `profiles.yml`:

```yaml
default:
  outputs:
    dev:
      type: postgres
      host: localhost
      user: myuser
      pass: mypass
      port: 5432
      dbname: mydb
      schema: my_schema
      threads: 4
      keepalives_idle: 0
  target: dev
```

Neste exemplo, temos um perfil chamado `dev` que se conecta a um banco de dados `PostgreSQL` local. Você pode adicionar quantos perfis quiser, e cada perfil pode se conectar a um banco de dados diferente.

Outro fator importante é a questão de segurança. Você não deve armazenar informações sensíveis, como senhas, diretamente no arquivo `profiles.yml`. Para isso, você pode utilizar variáveis de ambiente. Por exemplo, você pode substituir o valor da chave `pass` por uma variável de ambiente chamada `DBT_PASS`:

```yaml
default:
  outputs:
    dev:
      type: postgres
      host: localhost
      user: myuser
      pass: "{{ env_var('DBT_PASS') }}"
      port: 5432
      dbname: mydb
      schema: my_schema
      threads: 4
      keepalives_idle: 0
  target: dev
```

Dessa forma, você pode definir a variável de ambiente `DBT_PASS` no seu terminal antes de executar o dbt:

```bash
export DBT_PASS=mypass
```
No Windows, você pode definir variáveis de ambiente da seguinte maneira:

```bash
set DBT_PASS=mypass
```
Outra maneira de definir variáveis de ambiente é criar um arquivo `.env` na raiz do seu projeto e adicionar as variáveis de ambiente nele. O dbt irá ler automaticamente as variáveis de ambiente definidas no arquivo `.env`.

Exemplo de arquivo `.env`:
  
  ```bash 
  DBT_PASS=mypass
  ```

## Inicializando um projeto dbt

Para inicializar um projeto dbt, você pode executar o seguinte comando:

```bash
dbt init my_project
```
Este comando irá criar uma estrutura de pastas e arquivos para o seu projeto dbt. A estrutura de pastas e arquivos criada é a seguinte:

```text
my_project/
|-- analyses
|-- macros
|-- models
|-- seeds
|-- snapshots
|-- tests
|-- .env
|-- .gitignore
|-- dbt_project.yml
|-- profiles.yml
```
Aqui está uma breve descrição de cada pasta e arquivo:

- `analyses`: Pasta onde você pode escrever consultas SQL que serão executadas pelo dbt. As consultas nesta pasta são consultas ad-hoc, ou seja, consultas que você executa manualmente.
- `macros`: Pasta onde você pode escrever macros, que são consultas SQL reutilizáveis.
- `models`: Pasta onde você escreve as transformações de dados. Cada arquivo nesta pasta é um modelo dbt.
- `seeds`: Pasta onde você pode colocar arquivos estáticos com dados de origem - por exemplo - lista de municipios, status, ...
- `snapshots`: Pasta onde você pode escrever consultas SQL que criam snapshots de tabelas.
- `tests`: Pasta onde você pode escrever testes para os modelos dbt.
- `.env`: Arquivo onde você pode definir variáveis de ambiente.
- `.gitignore`: Arquivo que contém os arquivos e pastas que você deseja ignorar no Git.
- `dbt_project.yml`: Arquivo de configuração do projeto dbt.
- `profiles.yml`: Arquivo de configuração de conexão com o banco de dados.

## dbt_project.yml

O arquivo `dbt_project.yml` é o arquivo de configuração do projeto dbt. Este arquivo contém informações sobre o projeto, como o nome do projeto, a versão do projeto, o diretório onde os modelos dbt estão localizados, etc. Aqui está um exemplo de um arquivo `dbt_project.yml`:

```yaml
name: 'pipeline'
version: '1.0.0'

# This setting configures which "profile" dbt uses for this project.
profile: 'pipeline'

model-paths: ["models"]
analysis-paths: ["analyses"]
test-paths: ["tests"]
seed-paths: ["seeds"]
macro-paths: ["macros"]
snapshot-paths: ["snapshots"]

clean-targets:         # directories to be removed by `dbt clean`
  - "target"
  - "dbt_packages"

models:
  pipeline:
    # Config indicated by + and applies to all files under models/example/
    bronze:
      schema: bronze
      +materialized: table
    silver:
      schema: silver
      +materialized: table
    gold:
      schema: gold
      +materialized: table
      +post-hook: "{{ grant_and_notify(this) }}"
    src:
      schema: src
      +materialized: table
seeds:
  src:
  +schema: src
  +materialized: seed
```

Neste exemplo, temos um projeto chamado `pipeline` com três modelos: `bronze`, `silver` e `gold`. Cada modelo tem um esquema associado e um tipo de materialização. O tipo de materialização pode ser `table`, `view` ou `incremental`. O tipo de materialização `table` é o tipo de materialização padrão, e é usado para criar tabelas a partir de consultas SQL. O tipo de materialização `view` é usado para criar visualizações a partir de consultas SQL. O tipo de materialização `incremental` é usado para criar tabelas incrementais a partir de consultas SQL.

Este arquivo tem mais itens que são bem interessantes de aprender, e ao longo desta série de artigos, vamos voltar e abordar cada um deles.


## Escrita de modelos dbt

Vamos ao ponto principal do dbt: a escrita de modelos dbt. Um modelo dbt é um arquivo `.sql` que contém uma consulta SQL que transforma os dados. Aqui está um exemplo de um modelo dbt:

```sql
-- bronze/my_model.sql
with my_cte as (
    select
        column1,
        column2
    from
        my_table
)
select
    column1,
    column2
from
    my_cte
```

Este modelo dbt é um modelo `table`, que cria uma tabela chamada `my_model` no esquema `bronze`. O modelo dbt contém uma consulta SQL que seleciona as colunas `column1` e `column2` da tabela `my_table` e as insere na tabela `my_model`.



## Executando o dbt

Agore que você já tem um projeto dbt configurado e um modelo dbt escrito, você pode executar o dbt para criar as tabelas no banco de dados. Para executar o dbt, você pode usar o seguinte comando:

```bash
dbt run
```
Este comando irá executar todos os modelos dbt no diretório `models` e criar as tabelas no banco de dados. Se você quiser executar um modelo dbt específico, você pode usar o seguinte comando:

```bash
dbt run --models my_model
```
Este comando irá executar apenas o modelo `my_model`.


## Conclusão

Neste artigo, você aprendeu o que é o dbt, como instalar o dbt, como configurar o dbt, como inicializar um projeto dbt, como escrever modelos dbt e como executar o dbt. Nos próximos artigos, vamos falar em maiores detalhes sobre as funcionalidades e começar efetivamente a criar o nosso repositório de dados.